In [1]:
import numpy as np
import pickle
from numba import njit

In [33]:
# main code for simulating ATPase motor
@njit
def simulation(frame_num,ep_apo,ep,ep_bias,eb_atp,eb_adp,e_act,e_sub,k_c,k_hs,k_h2,k_norm,k_abnorm,c_atp,c_adp,c_atpgs,c_pi):

    kT=0.596
    k_on=1e5

    C_APO=0
    C_ATP=1
    C_ADP=2
    C_ATPGS=3
    C_NONACT=0
    C_ACT=1
    C_DISENGAGE=0
    C_ENGAGE=1
    C_OPEN=0
    C_CLOSE=1

    valid_engages=np.array(
        [[0,1,1,1,1,1],
        [0,0,1,1,1,1],
        [0,0,1,1,1,1],
        [0,0,0,1,1,1],
        [0,0,0,1,1,1]],dtype=np.intc)
    valid_surfaces=np.array(
        [[0,1,1,1,1,0],
        [1,0,1,1,1,0],
        [0,0,1,1,1,0],
        [1,0,0,1,1,0],
        [0,1,0,1,1,0]],dtype=np.intc)
    valid_heights=np.array(
        [[7,5,4,3,2,1],
        [7,7,4,3,2,1],
        [7,7,4,3,2,1],
        [7,7,7,3,2,1],
        [7,7,7,3,2,1]],dtype=np.intc)
    valid_jumps=[
        [(0,1), (0,-1), (1,1),  (1,-2), (2,0),  (2,-1), (3,-2), (4,0)],
        [(0,2), (0,-1), (1,2),  (1,-2), (3,0),  (4,-1)],
        [(0,1), (0,0),  (2,1),  (2,-1), (3,-2), (4,1)],
        [(0,2), (1,0),  (2,2),  (4,2),  (4,-1)],
        [(0,0), (1,1),  (2,-1), (3,-2), (3,1)]]

    temps=[]
    for conform_type1,valid_jump in enumerate(valid_jumps):
        heights1=np.roll(valid_heights[conform_type1],0)
        temp=[]
        for conform_type2,conform_shift2 in valid_jump:
            heights2=np.roll(valid_heights[conform_type2],conform_shift2)
            mask=np.logical_and(heights1!=7,heights2!=7)
            d_step=np.mean(heights1[mask]-heights2[mask])
            if np.isnan(d_step):
                d_step=0
            temp.append((conform_type2,conform_shift2,int(d_step)))
        temps.append(temp)
    valid_jumps=temps

    valid_jump_nums=np.array([len(i) for i in valid_jumps])

    # initial value
    nucleos=np.array([1,1,1,1,2,0],dtype=np.intc)
    activates=np.array([0,0,0,0,0,0],dtype=np.intc)
    conform_type=0
    conform_shift=-1
    engages=np.roll(valid_engages[conform_type],conform_shift)
    surfaces=np.roll(valid_surfaces[conform_type],conform_shift)
    is_abnormal=False
    t=0
    step=0
    hydr=0

    # for recording data
    t_all=np.zeros(frame_num)
    step_all=np.zeros(frame_num,dtype=np.intc)
    hydr_all=np.zeros(frame_num,dtype=np.intc)
    conform_distr=np.zeros(6)

    # main loop
    for frame in range(frame_num):

        # record data
        t_all[frame]=t
        step_all[frame]=step
        hydr_all[frame]=hydr

        nucleo_f_rates=np.zeros(6)
        nucleo_b_rates=np.zeros(6)
        atpgs_f_rates=np.zeros(6)
        atpgs_b_rates=np.zeros(6)
        conform_rates=np.zeros(valid_jump_nums[conform_type])

        if not is_abnormal:

            for i in range(6):
                nucleo1=nucleos[i]
                engage1=engages[i]
                surface1=surfaces[i]

                nucleo_f_rate=0
                nucleo_b_rate=0
                atpgs_f_rate=0
                atpgs_b_rate=0

                # hydrolysis rates
                if nucleo1==C_ATP:
                    # ATP hydrolysis rate
                    if surface1==C_CLOSE and engage1==C_ENGAGE and \
                    (surfaces[(i+1)%6]==C_OPEN or (surfaces[(i+1)%6]==C_CLOSE and surfaces[(i+2)%6]==C_OPEN)):
                        nucleo_f_rate=k_hs[i]
                    else:
                        nucleo_f_rate=0

                    # ATP dissociation rate
                    if surface1==C_OPEN and engage1==C_DISENGAGE:
                        de=ep_apo-0
                    else:
                        de=ep-0
                    if surface1!=C_OPEN:
                        de+=eb_atp-0
                    nucleo_b_rate=k_on*np.exp(-de/kT)

                elif nucleo1==C_ADP:
                    # ADP dissociation rate
                    if surface1==C_OPEN and engage1==C_DISENGAGE:
                        de=ep_apo-0
                    else:
                        de=ep-0
                    if surface1!=C_OPEN:
                        de+=eb_adp-0
                    nucleo_f_rate=k_on*np.exp(-de/kT)

                    # ATP synthetize rate
                    nucleo_b_rate=0

                elif nucleo1==C_APO:
                    # ATP binding rate
                    nucleo_f_rate=k_on*c_atp

                    # ADP binding rate
                    nucleo_b_rate=k_on*c_adp

                    # ATPgS binding rate
                    atpgs_f_rate=k_on*c_atpgs

                elif nucleo1==C_ATPGS:
                    # ATPgS dissociation rate
                    if surface1==C_OPEN and engage1==C_DISENGAGE:
                        de=ep_apo-0
                    else:
                        de=ep-0
                    if surface1!=C_OPEN:
                        de+=eb_atp-0
                    atpgs_b_rate=k_on*np.exp(-de/kT)

                nucleo_f_rates[i]=nucleo_f_rate
                nucleo_b_rates[i]=nucleo_b_rate
                atpgs_f_rates[i]=atpgs_f_rate
                atpgs_b_rates[i]=atpgs_b_rate
            
            # allosteric rates
            # first sum up the conformational energy
            de=0
            for i in range(6):
                if nucleos[i]!=C_APO:
                    if surfaces[i]==C_OPEN and engages[i]==C_DISENGAGE:
                        de+=ep_apo+ep_bias[i]
                    else:
                        de+=ep

                if surfaces[i]==C_CLOSE:
                    if nucleos[i]==C_ATP or nucleos[i]==C_ATPGS:
                        de+=eb_atp
                    elif nucleos[i]==C_ADP:
                        de+=eb_adp
                    if activates[i]==C_ACT:
                        de-=e_act

                elif surfaces[i]==C_OPEN:
                    if activates[i]==C_ACT and engages[(i+1)%6]==C_ENGAGE:
                        de-=e_act

            # then sum up the energy for each possible allosteric state and calculate the reaction rate
            valid_jump=valid_jumps[conform_type]
            for jump_idx in range(valid_jump_nums[conform_type]):
                conform_type_next,conform_shift_bias,d_step=valid_jump[jump_idx]
                engages_next=np.roll(valid_engages[conform_type_next],conform_shift+conform_shift_bias)
                surfaces_next=np.roll(valid_surfaces[conform_type_next],conform_shift+conform_shift_bias)
                de_next=0
                for i in range(6):
                    if nucleos[i]!=C_APO:
                        if surfaces_next[i]==C_OPEN and engages_next[i]==C_DISENGAGE:
                            de_next+=ep_apo+ep_bias[i]
                        else:
                            de_next+=ep

                    if surfaces_next[i]==C_CLOSE:
                        if nucleos[i]==C_ATP or nucleos[i]==C_ATPGS:
                            de_next+=eb_atp
                        elif nucleos[i]==C_ADP:
                            de_next+=eb_adp
                        if activates[i]==C_ACT:
                            de_next-=e_act

                    elif surfaces_next[i]==C_OPEN:
                        if activates[i]==C_ACT and engages_next[(i+1)%6]==C_ENGAGE:
                            de_next-=e_act

                de_next-=d_step*e_sub
                conform_rate=k_c*np.exp((de_next-de)/kT/2)
                conform_rates[jump_idx]=conform_rate

        if is_abnormal:
            norm_rate=np.array([0.,k_norm,k_h2*np.sum(nucleos==C_ATP)/6])
        else:
            if np.all(np.logical_or(nucleos==C_ATP,nucleos==C_ATPGS)):
                norm_rate=np.array([k_abnorm,0.,0.])
            else:
                norm_rate=np.array([0.,0.,0.])

        # gillespie algorithm
        rates=np.hstack((nucleo_f_rates,nucleo_b_rates,atpgs_f_rates,atpgs_b_rates,norm_rate,conform_rates))

        conform_type_temp=conform_type
        conform_shift_temp=conform_shift

        total_rate=np.sum(rates)
        random=np.random.rand()*total_rate
        rates_cum=np.cumsum(rates)
        var_id=np.sum(rates_cum<random)

        if var_id>=0 and var_id<6:
            nucleos[var_id%6]=(nucleos[var_id%6]+1)%3
            if nucleos[var_id%6]==C_ADP:
                activates[var_id%6]=C_ACT
                hydr+=1

        elif var_id>=6 and var_id<12:
            nucleos[var_id%6]=(nucleos[var_id%6]-1)%3

        elif var_id>=12 and var_id<18:
            nucleos[var_id%6]=3

        elif var_id>=18 and var_id<24:
            nucleos[var_id%6]=0

        elif var_id==24:
            is_abnormal=True

        elif var_id==25:
            is_abnormal=False

        elif var_id==26:
            hydr+=1

        elif var_id>=27:
            idx=var_id-27
            conform_type_next,conform_shift_bias,d_step=valid_jumps[conform_type][idx]
            conform_shift_next=(conform_shift+conform_shift_bias)%6
            engages_next=np.roll(valid_engages[conform_type_next],conform_shift_next)
            surfaces_next=np.roll(valid_surfaces[conform_type_next],conform_shift_next)

            for i in range(6):
                if activates[i]==C_ACT:
                    if surfaces[i]==C_CLOSE and surfaces_next[i]==C_OPEN:
                            activates[i]=C_NONACT

            conform_type=conform_type_next
            conform_shift=conform_shift_next
            engages=engages_next
            surfaces=surfaces_next
            if not is_abnormal:
                step+=d_step

        dt=np.random.exponential(scale=1/total_rate)
        t+=dt

        if conform_type_temp==0:
            conform_distr[conform_shift_temp]+=dt

    return t_all,step_all,hydr_all,conform_distr

In [34]:
# default parameters
ep_apo=3.7
ep=7.4
ep_bias=np.array([0.0,0.0,0.0,0.0,0.0,0.0])

eb_atp=2.1
eb_adp=0.62

e_act=3.7
e_sub=0.1

k_c=0.8
k_hs=np.ones(6)*3.2
k_h2=4.5

k_norm=0.6
k_abnorm=7.0

args=(ep_apo,ep,ep_bias,eb_atp,eb_adp,e_act,e_sub,k_c,k_hs,k_h2,k_norm,k_abnorm)

In [4]:
# calculate the translocation rate and hydrolysis rate under different ATP concentrations
repeat=10
frame_num=500000
c_atps=np.geomspace(3e-6,3e-2,20)

trans_atp=[]
hydr_atp=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for c_atp in c_atps:
        t_all,step_all,hydr_all,_=simulation(frame_num,*args,c_atp,0,0,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_atp.append(trans_single)
    hydr_atp.append(hydr_single)

trans_atp=np.array(trans_atp)
hydr_atp=np.array(hydr_atp)

In [5]:
# calculate the translocation rates and hydrolysis rates under different ADP concentrations
repeat=10
frame_num=500000
c_adps=np.geomspace(2e-5,10e-3,20)

trans_adp=[]
hydr_adp=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for c_adp in c_adps:
        t_all,step_all,hydr_all,_=simulation(frame_num,*args,5e-4,c_adp,0,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_adp.append(trans_single)
    hydr_adp.append(hydr_single)

trans_adp=np.array(trans_adp)
hydr_adp=np.array(hydr_adp)

In [6]:
# calculate the translocation rates and hydrolysis rates under different ATPgS concentrations and a lower ATP concentration
repeat=10
frame_num=500000
c_atpgss=np.geomspace(2e-6,1.5e-4,20)

trans_atpgs=[]
hydr_atpgs=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for c_atpgs in c_atpgss:
        t_all,step_all,hydr_all,_=simulation(frame_num,*args,5e-4,0,c_atpgs,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_atpgs.append(trans_single)
    hydr_atpgs.append(hydr_single)

trans_atpgs=np.array(trans_atpgs)
hydr_atpgs=np.array(hydr_atpgs)

In [7]:
# calculate the translocation rates and hydrolysis rates under different ATPgS concentrations and a higher ATP concentration
repeat=10
frame_num=500000
c_atpgss2=np.geomspace(2e-5,4e-3,20)

trans_atpgs2=[]
hydr_atpgs2=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for c_atpgs in c_atpgss2:
        t_all,step_all,hydr_all,_=simulation(frame_num,*args,15e-3,0,c_atpgs,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_atpgs2.append(trans_single)
    hydr_atpgs2.append(hydr_single)

trans_atpgs2=np.array(trans_atpgs2)
hydr_atpgs2=np.array(hydr_atpgs2)

In [ ]:
# calculate the translocation rates and hydrolysis rates under different ADP concentrations for substrate without FA
e_sub_temp=0.2
args_temp=(ep_apo,ep,ep_bias,eb_atp,eb_adp,e_act,e_sub_temp,k_c,k_hs,k_h2,k_norm,k_abnorm)

repeat=10
frame_num=500000
c_atps_lobs=np.geomspace(3e-6,3e-2,20)

trans_atp_lobs=[]
hydr_atp_lobs=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for c_atp in c_atps_lobs:
        t_all,step_all,hydr_all,_=simulation(frame_num,*args_temp,c_atp,0,0,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_atp_lobs.append(trans_single)
    hydr_atp_lobs.append(hydr_single)

trans_atp_lobs=np.array(trans_atp_lobs)
hydr_atp_lobs=np.array(hydr_atp_lobs)

In [ ]:
# calculate the translocation rates and hydrolysis rates under different ADP concentrations for substrate without FA
e_sub_temp=0.65
args_temp=(ep_apo,ep,ep_bias,eb_atp,eb_adp,e_act,e_sub_temp,k_c,k_hs,k_h2,k_norm,k_abnorm)

repeat=10
frame_num=500000
c_atps_hobs=np.geomspace(3e-6,3e-2,20)

trans_atp_hobs=[]
hydr_atp_hobs=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for c_atp in c_atps_hobs:
        t_all,step_all,hydr_all,_=simulation(frame_num,*args_temp,c_atp,0,0,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_atp_hobs.append(trans_single)
    hydr_atp_hobs.append(hydr_single)

trans_atp_hobs=np.array(trans_atp_hobs)
hydr_atp_hobs=np.array(hydr_atp_hobs)

In [8]:
# calculate the translocation rates and hydrolysis rates under different external forces
repeat=10
frame_num=500000
e_subs=np.linspace(0,13*0.144*0.7*1.5,20)

trans_force=[]
hydr_force=[]
for i in range(repeat):
    t_curve=[]
    step_curve=[]
    hydr_curve=[]
    for e_sub_temp in e_subs:
        args_temp=(ep_apo,ep,ep_bias,eb_atp,eb_adp,e_act,e_sub_temp,k_c,k_hs,k_h2,k_norm,k_abnorm)
        t_all,step_all,hydr_all,_=simulation(frame_num,*args_temp,1e-3,0,0,0)
        t_curve.append(t_all[-1])
        step_curve.append(step_all[-1])
        hydr_curve.append(hydr_all[-1])

    t_curve=np.array(t_curve)
    step_curve=np.array(step_curve)
    hydr_curve=np.array(hydr_curve)
    trans_single=step_curve/t_curve
    hydr_single=hydr_curve/t_curve
    trans_force.append(trans_single)
    hydr_force.append(hydr_single)

trans_force=np.array(trans_force)
hydr_force=np.array(hydr_force)

In [20]:
# save the reselts
with open('result_new.dat','wb') as f:
    data=(c_atps,trans_atp,hydr_atp,
          c_adps,trans_adp,hydr_adp,
          c_atpgss,trans_atpgs,hydr_atpgs,
          c_atpgss2,trans_atpgs2,hydr_atpgs2,
          c_atps_lobs,trans_atp_lobs,hydr_atp_lobs,
          c_atps_hobs,trans_atp_hobs,hydr_atp_hobs,
          e_subs,trans_force,hydr_force)
    pickle.dump(data,f)